In [1]:
import torch
import torch.nn as nn
import numpy as np
import math

import os
import sys
root = os.path.abspath(os.curdir)
sys.path.append(root)

device = 'cuda:0'

In [2]:
# testing independent clip
bs = 2
in_ch = 5
out_ch = 5
k = 2

x = torch.randn(bs,in_ch).to(device)
weight = torch.randn(out_ch, in_ch).to(device) 

print(x)
print(weight)

tensor([[ 0.9640,  0.1733, -0.0119, -0.6839, -0.1156],
        [-1.2953, -1.2084, -0.4730, -1.7243, -0.2829]], device='cuda:0')
tensor([[-0.9351,  1.0108, -0.2260,  0.6809, -0.0114],
        [ 1.8429,  1.3490,  0.0785, -1.0828, -1.1014],
        [ 0.0106,  0.4968,  0.6381, -0.1247, -2.0208],
        [-0.3257,  1.2431, -1.0835, -0.7979,  0.6470],
        [ 0.6788,  0.5792,  1.6970,  0.0283,  1.4778]], device='cuda:0')


In [36]:
# regular ind trunc

print(x)
x_vals = x.clone().detach() #(bs, in_features)
print('x shape: ',x_vals.shape)
temp = (x_vals.view(-1,1,in_ch)*weight).sum(axis=1) # (bs, out_features, in_features)
print('sum of x@W along input pixel dimension: ',temp.shape)
# x = torch.matmul(x,weight.T) #(bs, out_features)
# print('x@W: ',x.shape)
# print(temp.sum(axis=1))
# sum over last axis gives you x@W
# sum over first axis gives you largest contribution per in_feature
# for x in bs, out_features element wise prod. of w and x
_, idx_top = torch.topk(temp,k) #(bs, out_dim, self.k)
_, idx_bot = torch.topk(-1*temp,k)
print(idx_top)
print(idx_bot)
z = torch.ones(bs,in_ch).to(device)
z[np.arange(bs),idx_top.T] = 0
z[np.arange(bs),idx_bot.T] = 0
print(z)
# print(x)
x = torch.matmul(z*x,weight.T)
# print(z*x)
print(x)

# print(val_top.shape)

# x -= val_top.sum(axis=-1)
# x += val_bot.sum(axis=-1)


tensor([[-1.2630, -0.7869,  1.3594,  0.8437, -1.0029],
        [-1.2470,  2.0593, -1.3713,  1.3699,  1.2348]], device='cuda:0')
x shape:  torch.Size([2, 5])
sum of x@W along input pixel dimension:  torch.Size([2, 5])
tensor([[2, 3],
        [1, 3]], device='cuda:0')
tensor([[0, 4],
        [2, 0]], device='cuda:0')
tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.]], device='cuda:0')
tensor([[-0.7869, -0.7869, -0.7869, -0.7869, -0.7869],
        [ 1.2348,  1.2348,  1.2348,  1.2348,  1.2348]], device='cuda:0')


In [43]:
torch.abs(torch.randn(5))

tensor([0.8574, 0.4394, 0.7596, 1.1383, 1.8523])

In [42]:
z = torch.ones(bs,in_ch).to(device)
print(z)
print(idx_top)
# print(z[np.arange(bs),idx_top.T])
z[:,idx_top.T] = 0
print(z)


tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]], device='cuda:0')
tensor([[2, 3],
        [1, 3]], device='cuda:0')
tensor([[1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.]], device='cuda:0')


In [38]:
idx_top[np.arange]

IndexError: only integers, slices (`:`), ellipsis (`...`), None and long or byte Variables are valid indices (got builtin_function_or_method)

In [ ]:
w,h = 28,28
kernel_dim = 3
bs = 128
in_ch = 3
out_ch = 3

image = torch.randn(bs,in_ch,w,h).to(device)
kernel = torch.randn(out_ch,in_ch,kernel_dim,kernel_dim).to(device)

In [4]:
nn_conv = torch.nn.Conv2d(in_ch,out_ch,kernel_dim,bias=True).to(device)

conv = my_conv(in_ch,out_ch,kernel_dim,bias=True).to(device)
conv.weight = nn_conv.weight
conv.bias = nn_conv.bias

In [5]:
nn_out = nn_conv(image)
out = conv(image)

In [6]:
print(nn_out.shape)
print(out.shape)
print(((nn_out - out)<0.00001).sum() == np.prod(nn_out.shape))
# print(((out - my_out)))

torch.Size([128, 3, 26, 26])
torch.Size([128, 3, 26, 26])
tensor(True, device='cuda:0')
